# Process Globe-LFMC samples and extract DEM Data
Extracts the samples for locations in the CONUS from the Globe-LFMC spread sheet and adds the normalised DEM and other auxiliary data. DEM data is from the GEE SRTM DEM product, extracted using the MODIS projection and scale. Sites within the same MODIS pixel are merged. The following files are created:
- `LFMC_CONUS.csv`: CONUS data extracted from the Globe-LFMC dataset
- `LFMC_sites.csv`: sites extracted from the Globe-LFMC CONUS data
- `LFMC_sites_dem.csv`: sites augmented with normalised DEM and location data
- `LFMC_samples_dem.csv`: Globe-LFMC CONUS sample data augmented with auxiliary variables

### Notes
1. The `Globe-LFMC-v2.xlsx` should exist in the `DATA_DIR` directory
2. The sample data output by this code is further processed by the MODIS extraction code to remove the snow samples.

In [1]:
import os
import numpy as np
import pandas as pd
import time
from datetime import datetime
from datetime import timedelta

Define input and output files

In [2]:
# Directories
DATA_DIR = r"G:\My Drive\LFMC from MODIS\Globe-LFMC"

# Globe-LFMC file and sheet name
GLOBE_LFMC = os.path.join(DATA_DIR, "Globe-LFMC-v2.xlsx")
SHEET_NAME = "LFMC data"

# File Names
LFMC_RAW = os.path.join(DATA_DIR, "LFMC_CONUS.csv")            # CSV of CONUS data extracted from the Globe-LFMC dataset
LFMC_SITES = os.path.join(DATA_DIR, "LFMC_sites.csv")          # Output file: sites extracted from the Globe-LFMC dataset
LFMC_SITES_DEM = os.path.join(DATA_DIR, "LFMC_sites_dem.csv")  # Output file: sites augmented with normalised DEM and location data
DEM_OUTPUT = os.path.join(DATA_DIR, "LFMC_samples_dem.csv")    # Output file: Globe-LFMC CONUS sample data augmented with auxiliary variables

Other constants/parameters

In [3]:
START_DATE = "2002-01-01"
END_DATE = "2019-01-01"

# DEM Product, projection and resolution
DEM_PRODUCT = 'USGS/SRTMGL1_003'
DEM_PROJ = "EPSG:4326"
DEM_SCALE = 30

# MODIS projection and resolution
MODIS_PROJ = "SR-ORG:6974"
MODIS_SCALE = 463.3127

# Floating point precision
FLOAT_PRE = 5

Initialise Google Earth Engine

In [4]:
import ee
ee.Initialize()

## Normalise data
- data: data to be normalised - a Pandas series, numpy array or similar
- in_range: range of source data; calculated from the source data if not specified
- out_range: range of normalised data; range is 0-1 if not specified

In [5]:
def normalise(data, in_range=None, out_range=(0,1)):
    if in_range is None:
        in_range = (data.min(), data.max())
    temp = (data - in_range[0]) / (in_range[1] - in_range[0])
    if out_range == (0,1):
        return temp
    else:
        return temp * (out_range[1] - out_range[0]) + out_range[0]

## Extract DEM data
Extracts the DEM data at the requested projection and resolution. Terrain.products adds the slope and aspect. A reducer is used so terrain product info is added before resampling.
- Parameters:
 - sites: dataframe of sampling sites
 - scale/proj: the required scale/proj (e.g. MODIS scale/proj - or map scale/proj)
 - maxPixels: Reducer parameter specifying the maximum number of DEM pixels to use to compute each down-sampled pixel. Doesn't need to be exact but make sure it's large enough - 512 is good for MODIS
- Returns: Dataframe of sites with latitude and longitude set to the pixel centroid as returned by GEE and the added elevation/slope/aspect attributes

In [6]:
def get_dem_data(sites, scale, proj, max_pixels):
    dem_image = ee.Terrain.products(ee.Image(DEM_PRODUCT)).reduceResolution(ee.Reducer.mean(), maxPixels=max_pixels)
    points = [ee.Geometry.Point(site.Longitude, site.Latitude) for x, site in sites.iterrows()]
    dem_col = ee.ImageCollection(dem_image)
    col_list = [dem_col.getRegion(point, scale, proj) for point in points]
    dem_list = ee.List(col_list).getInfo()
    dem_data = pd.DataFrame([item[1] for item in dem_list], columns=dem_list[0][0])
    dem_data.id = sites.Site
    dem_data.columns = ['Site', 'Longitude', 'Latitude', 'time', 'Elevation', 'Slope', 'Aspect', 'hillshade']
    dem_df = dem_data.drop(columns=["time", "hillshade"]).\
        round({'Elevation': 0, 'Slope': 0, 'Aspect': 0}).\
        astype({'Elevation': 'int32', 'Slope': 'int32', 'Aspect': 'int32'})
    return dem_df

## Main Processing
Extract Globe LFMC data from the excel workbook sheet and save to the LFMC_RAW file. If the file has previously been created, skip this step and run the next one.

In [7]:
LFMC_data = pd.read_excel(GLOBE_LFMC, SHEET_NAME).dropna(how="all")
LFMC_data = LFMC_data[(LFMC_data.Country == "USA")
                      & (LFMC_data["State/Region"] != "Alaska")
                      & (LFMC_data["Sampling date"] >= START_DATE)]
LFMC_data.to_csv(LFMC_RAW)
LFMC_data = LFMC_data.astype(dtype={'Sampling year': np.int32, 'Protocol': np.int16, 'Land Cover': np.int32, 'Units': np.int16})
LFMC_data

,ID,Contact,Sitename,State/Region,Country,Latitude,Longitude,Sampling time,Sampling date,Sampling year,...,Units,NDVI SD min,NDVI SD max,NDVI CV min,NDVI CV max,Species collected,Elevation(m.a.s.l),Slope(%),Reference,Name of picture file
3675,C4_1_1,Dennison,Rush Valley A,Utah,USA,40.2143,-112.21748,12:33:00,2005-06-20,2005,...,1,0.014140,0.031859,0.076630,0.138643,Artemisia tridentata,1573.0,0,NaN,NaN
3676,C4_1_2,Dennison,Rush Valley A,Utah,USA,40.2143,-112.21748,14:07:00,2005-07-05,2005,...,1,0.014140,0.031859,0.076630,0.138643,Artemisia tridentata,1573.0,0,NaN,NaN
3677,C4_1_3,Dennison,Rush Valley A,Utah,USA,40.2143,-112.21748,10:09:00,2005-07-21,2005,...,1,0.014140,0.031859,0.076630,0.138643,Artemisia tridentata,1573.0,0,NaN,NaN
3678,C4_1_4,Dennison,Rush Valley A,Utah,USA,40.2143,-112.21748,10:04:00,2005-08-08,2005,...,1,0.014140,0.031859,0.076630,0.138643,Artemisia tridentata,1573.0,0,NaN,NaN
3679,C4_1_5,Dennison,Rush Valley A,Utah,USA,40.2143,-112.21748,09:51:00,2005-08-23,2005,...,1,0.014140,0.031859,0.076630,0.138643,Artemisia tridentata,1573.0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161132,C13_4_14,Qi,Missoula 4,Montana,USA,46.8973,-113.43560,12:00:00,2012-08-28,2012,...,1,0.063012,0.282592,0.114161,0.318335,Artemisia tridentata,1250.0,NaN,"Qi, Yi; Dennison, Philip E.; Jolly, W. Matt; K...",sagebrush site YQ.jpg
161133,C13_4_15,Qi,Missoula 4,Montana,USA,46.8973,-113.43560,12:00:00,2012-09-04,2012,...,1,0.063012,0.282592,0.114161,0.318335,Artemisia tridentata,1250.0,NaN,"Qi, Yi; Dennison, Philip E.; Jolly, W. Matt; K...",sagebrush site YQ.jpg
161134,C13_4_16,Qi,Missoula 4,Montana,USA,46.8973,-113.43560,12:00:00,2012-09-11,2012,...,1,0.063012,0.282592,0.114161,0.318335,Artemisia tridentata,1250.0,NaN,"Qi, Yi; Dennison, Philip E.; Jolly, W. Matt; K...",sagebrush site YQ.jpg
161135,C13_4_17,Qi,Missoula 4,Montana,USA,46.8973,-113.43560,12:00:00,2012-09-18,2012,...,1,0.063012,0.282592,0.114161,0.318335,Artemisia tridentata,1250.0,NaN,"Qi, Yi; Dennison, Philip E.; Jolly, W. Matt; K...",sagebrush site YQ.jpg


If the LFMC_RAW file already exists, uncomment and run the following cell instead of the previous one.

In [8]:
#LFMC_data = pd.read_csv(LFMC_RAW, index_col=0, float_precision="high", parse_dates=["Sampling date"],
#                        dtype={8: str, 10: np.int32, 11: np.int16, 12: np.int32, 14: np.int16, 23: str})
#LFMC_data

### Site processing
Extract the unique sites from the Globe-LFMC data

In [9]:
LFMC_data["Site"] = LFMC_data.ID.str.rsplit("_", 1, expand=True)[0]
sites = LFMC_data[["Site", "Latitude", "Longitude"]].drop_duplicates().reset_index(drop=True)
sites.to_csv(LFMC_SITES, index=False)
sites

,Site,Latitude,Longitude
0,C4_1,40.214300,-112.217480
1,C4_2,40.213630,-112.213950
2,C4_3,40.210800,-112.207900
3,C4_4,40.211510,-112.212230
4,C4_5,40.212460,-112.212860
...,...,...,...
927,C6_939,34.965556,-108.586667
928,C13_1,46.633470,-114.576100
929,C13_2,46.735780,-114.536300
930,C13_3,46.923240,-113.440400


Retrieve the DEM data from GEE

In [10]:
dem_df = get_dem_data(sites, MODIS_SCALE, MODIS_PROJ, 512)
dem_df

,Site,Longitude,Latitude,Elevation,Slope,Aspect
0,C4_1,-112.218682,40.214582,1572,2,178
1,C4_2,-112.213226,40.214582,1577,2,173
2,C4_3,-112.206327,40.210415,1582,2,188
3,C4_4,-112.211783,40.210415,1576,2,187
4,C4_5,-112.211783,40.210415,1576,2,187
...,...,...,...,...,...,...
927,C6_939,-108.584153,34.964582,2320,6,192
928,C13_1,-114.583010,46.635415,1632,13,109
929,C13_2,-114.533901,46.735415,1327,14,100
930,C13_3,-113.439468,46.922915,1152,4,90


Normalise the DEM data and save the sites data. Note: sites with same latitude/longitude are *not* merged yet

In [11]:
dem_norm = dem_df[["Site", "Longitude", "Latitude"]].round(FLOAT_PRE)
longitude = normalise(dem_df.Longitude, (-180, 180), (-np.pi, np.pi))
dem_norm["Long_sin"] = longitude.transform(np.sin).round(FLOAT_PRE)
dem_norm["Long_cos"] = longitude.transform(np.cos).round(FLOAT_PRE)
dem_norm["Lat_norm"] = normalise(dem_df.Latitude, (-90, 90)).round(FLOAT_PRE)
dem_norm["Elevation"] = normalise(dem_df.Elevation, (0, 6000)).round(FLOAT_PRE)
dem_norm["Slope"] = normalise(dem_df.Slope, (0, 90)).round(FLOAT_PRE)
aspect = normalise(dem_df.Aspect, (0, 360), (-np.pi, np.pi))
dem_norm["Aspect_sin"] = aspect.transform(np.sin).round(FLOAT_PRE)
dem_norm["Aspect_cos"] = aspect.transform(np.cos).round(FLOAT_PRE)
dem_norm.to_csv(LFMC_SITES_DEM, index=False)
dem_norm

,Site,Longitude,Latitude,Long_sin,Long_cos,Lat_norm,Elevation,Slope,Aspect_sin,Aspect_cos
0,C4_1,-112.21868,40.21458,-0.92575,-0.37814,0.72341,0.26200,0.02222,-0.03490,0.99939
1,C4_2,-112.21323,40.21458,-0.92578,-0.37805,0.72341,0.26283,0.02222,-0.12187,0.99255
2,C4_3,-112.20633,40.21042,-0.92583,-0.37794,0.72339,0.26367,0.02222,0.13917,0.99027
3,C4_4,-112.21178,40.21042,-0.92579,-0.37803,0.72339,0.26267,0.02222,0.12187,0.99255
4,C4_5,-112.21178,40.21042,-0.92579,-0.37803,0.72339,0.26267,0.02222,0.12187,0.99255
...,...,...,...,...,...,...,...,...,...,...
927,C6_939,-108.58415,34.96458,-0.94786,-0.31870,0.69425,0.38667,0.06667,0.20791,0.97815
928,C13_1,-114.58301,46.63541,-0.90936,-0.41601,0.75909,0.27200,0.14444,-0.94552,0.32557
929,C13_2,-114.53390,46.73541,-0.90972,-0.41523,0.75964,0.22117,0.15556,-0.98481,0.17365
930,C13_3,-113.43947,46.92291,-0.91748,-0.39778,0.76068,0.19200,0.04444,-1.00000,0.00000


### Date processing
Create dataframe with dates and normalised day-of-year

In [12]:
days = pd.date_range(START_DATE, END_DATE, closed="left")
doy = pd.Series(normalise(days.dayofyear, (1, 366), (-np.pi, np.pi)))
days_df = pd.DataFrame({"Date": days, 
                        "Day_sin": doy.transform(np.sin).round(FLOAT_PRE),
                        "Day_cos": doy.transform(np.cos).round(FLOAT_PRE)})
days_df

,Date,Day_sin,Day_cos
0,2002-01-01,-0.00000,-1.00000
1,2002-01-02,-0.01721,-0.99985
2,2002-01-03,-0.03442,-0.99941
3,2002-01-04,-0.05162,-0.99867
4,2002-01-05,-0.06880,-0.99763
...,...,...,...
6204,2018-12-27,0.08596,-0.99630
6205,2018-12-28,0.06880,-0.99763
6206,2018-12-29,0.05162,-0.99867
6207,2018-12-30,0.03442,-0.99941


### Sample processing
Create the auxiliary dataset from the samples

Step 1: Merge sites and sample data to add the site longitude and latitude to the samples

In [13]:
samples = dem_norm[["Site", "Longitude", "Latitude"]].merge(
    LFMC_data[["ID", "Site", "Sampling date", "Sampling year", "Land Cover", "LFMC value"]])
samples

,Site,Longitude,Latitude,ID,Sampling date,Sampling year,Land Cover,LFMC value
0,C4_1,-112.21868,40.21458,C4_1_1,2005-06-20,2005,120,156.763000
1,C4_1,-112.21868,40.21458,C4_1_2,2005-07-05,2005,120,128.277000
2,C4_1,-112.21868,40.21458,C4_1_3,2005-07-21,2005,120,92.482000
3,C4_1,-112.21868,40.21458,C4_1_4,2005-08-08,2005,120,82.093000
4,C4_1,-112.21868,40.21458,C4_1_5,2005-08-23,2005,120,78.953000
...,...,...,...,...,...,...,...,...
123150,C13_4,-113.43535,46.89791,C13_4_14,2012-08-28,2012,71,102.442066
123151,C13_4,-113.43535,46.89791,C13_4_15,2012-09-04,2012,71,88.764360
123152,C13_4,-113.43535,46.89791,C13_4_16,2012-09-11,2012,71,88.793816
123153,C13_4,-113.43535,46.89791,C13_4_17,2012-09-18,2012,71,81.723446


Step 2: Merge samples for same latitude/longitude/date

In [14]:
# Generate a common site id for each site with the same latitude and longitude
merge_columns = ["Latitude", "Longitude"]
sites_temp = dem_norm[merge_columns + ["Site"]].groupby(merge_columns, as_index=False).min()
# Merge samples for same year and location
samples = samples.merge(sites_temp, on=merge_columns, suffixes=("_x", None))
groupby_cols = ["Latitude", "Longitude", "Sampling date"]
data_cols = {"ID": "min",                                    # Unique sample ID is the first ID of the merged samples
             "Sampling year": "min",                         # They should all be the same, but need to select one
             "Land Cover": lambda x: pd.Series.mode(x)[0],   # Most common land cover value
             "LFMC value": "mean",                           # mean LFMC value
             "Site": "min"}                                  # Site id from sites_temp
samples = samples[groupby_cols + list(data_cols.keys())].groupby(groupby_cols, as_index=False).\
              agg(data_cols).round({"LFMC value": FLOAT_PRE})
samples

,Latitude,Longitude,Sampling date,ID,Sampling year,Land Cover,LFMC value,Site
0,25.99792,-97.57114,2017-06-15,C6_486_1,2017,80,147.00000,C6_486
1,25.99792,-97.57114,2017-07-11,C6_486_4,2017,80,202.66667,C6_486
2,25.99792,-97.57114,2017-07-15,C6_486_7,2017,80,345.00000,C6_486
3,25.99792,-97.57114,2017-08-15,C6_486_10,2017,80,233.75000,C6_486
4,25.99792,-97.57114,2017-09-21,C6_486_12,2017,80,191.00000,C6_486
...,...,...,...,...,...,...,...,...
66464,48.90208,-116.29750,2015-08-04,C6_91_11,2015,71,134.50000,C6_91
66465,48.90208,-116.29750,2015-08-12,C6_91_13,2015,71,115.00000,C6_91
66466,48.90208,-116.29750,2015-08-26,C6_91_15,2015,71,54.00000,C6_91
66467,48.90208,-116.29750,2015-09-10,C6_91_17,2015,71,79.50000,C6_91


Step 3: Add the normalised auxiliary variables (day-of-year, location and DEM) to the samples

In [15]:
aux_df = samples[["ID", "Latitude", "Longitude", "Sampling date", "Sampling year", "Land Cover", "LFMC value", "Site"]
                ].merge(days_df, left_on="Sampling date", right_on = "Date").drop(columns="Date").\
                merge(dem_norm.drop(columns=["Longitude", "Latitude"]), on="Site").sort_values("ID")
aux_df.to_csv(DEM_OUTPUT, index=False)
aux_df

,ID,Latitude,Longitude,Sampling date,Sampling year,Land Cover,LFMC value,Site,Day_sin,Day_cos,Long_sin,Long_cos,Lat_norm,Elevation,Slope,Aspect_sin,Aspect_cos
65903,C13_1_1,46.63541,-114.58301,2012-06-07,2012,70,97.59127,C13_1,-0.40936,0.91237,-0.90936,-0.41601,0.75909,0.27200,0.14444,-0.94552,0.32557
65910,C13_1_10,46.63541,-114.58301,2012-08-09,2012,70,139.17631,C13_1,0.61528,0.78831,-0.90936,-0.41601,0.75909,0.27200,0.14444,-0.94552,0.32557
65904,C13_1_11,46.63541,-114.58301,2012-08-16,2012,70,134.22227,C13_1,0.70558,0.70863,-0.90936,-0.41601,0.75909,0.27200,0.14444,-0.94552,0.32557
65907,C13_1_12,46.63541,-114.58301,2012-08-21,2012,70,126.68514,C13_1,0.76389,0.64535,-0.90936,-0.41601,0.75909,0.27200,0.14444,-0.94552,0.32557
65898,C13_1_13,46.63541,-114.58301,2012-08-30,2012,70,123.16282,C13_1,0.85432,0.51974,-0.90936,-0.41601,0.75909,0.27200,0.14444,-0.94552,0.32557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53631,C6_9_68,44.37292,-68.26178,2017-07-31,2017,70,127.20000,C6_9,0.47116,0.88205,-0.92889,0.37037,0.74652,0.01917,0.06667,0.85717,0.51504
53638,C6_9_73,44.37292,-68.26178,2017-08-07,2017,70,128.20000,C6_9,0.57377,0.81901,-0.92889,0.37037,0.74652,0.01917,0.06667,0.85717,0.51504
53621,C6_9_78,44.37292,-68.26178,2017-08-14,2017,70,128.60000,C6_9,0.66806,0.74410,-0.92889,0.37037,0.74652,0.01917,0.06667,0.85717,0.51504
53640,C6_9_83,44.37292,-68.26178,2017-09-13,2017,70,132.60000,C6_9,0.94836,0.31719,-0.92889,0.37037,0.74652,0.01917,0.06667,0.85717,0.51504
